# ReviewLens AI: Model Development and Validation

**Author:** Kevin Della Piazza
**Date:** October 2025

## 1. Introduction
This notebook documents the development and validation process for the core AI models of the ReviewLens platform. The objective is to test a suite of four distinct NLP models on a sample of the dataset to validate their effectiveness and to prototype the foundational Python code for the production AWS Lambda functions. This notebook serves as a professional artifact detailing the data science workflow, from data preparation to model evaluation.

### AI Models Tested:
1.  **Sentiment Analysis:** For overall positive/negative classification.
2.  **Zero-Shot Classification:** For dynamic topic tagging.
3.  **Aspect-Based Sentiment Analysis (ABSA):** For fine-grained sentiment on specific features.
4.  **Topic Modeling:** For discovering latent themes in the text corpus.

In [9]:
# --- Core Libraries ---
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import warnings
from IPython.display import display
import re
import stopit

# --- AI Libraries ---
from transformers import pipeline
from pyabsa import ATEPCCheckpointManager
from bertopic import BERTopic

# --- Validation ---
from sklearn.metrics import accuracy_score, classification_report

# --- Configuration ---
# Suppress ignorable warnings for a cleaner output
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
tqdm.pandas()

print("Libraries imported successfully.")

Libraries imported successfully.


## 2. Data Loading, Cleaning & Sanitization
We load the raw dataset and apply the cleaning steps that will be replicated in our AWS pipeline. A text sanitization step is included to prevent errors from special characters, making the code more robust for production.

In [10]:
# Load the raw dataset
try:
    # Use a relative path to ensure portability
    file_path = '../data/reviews.csv'
    df = pd.read_csv(file_path)
    print(f"Dataset loaded successfully with {len(df)} rows.")

    # --- Data Cleaning ---
    print("Starting data cleaning...")
    df_cleaned = df.drop('Unnamed: 0', axis=1, errors='ignore')
    df_cleaned.dropna(subset=['Review Text'], inplace=True)
    df_cleaned['Title'] = df_cleaned['Title'].fillna('')
    df_cleaned['full_review_text'] = df_cleaned['Title'] + ' ' + df_cleaned['Review Text']
    df_cleaned.dropna(subset=['Division Name', 'Department Name', 'Class Name'], inplace=True)
    
    # --- Text Sanitization Function ---
    def sanitize_text(text):
        if not isinstance(text, str): return ""
        # Replace ampersand, which was identified as a "poison pill" for ABSA
        text = text.replace('&', 'and')
        # Remove non-printable control characters
        text = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\x9f]', '', text)
        return text

    # Apply sanitization to the main text column
    df_cleaned['full_review_text'] = df_cleaned['full_review_text'].apply(sanitize_text)
    print(f"Data cleaned and sanitized. {len(df_cleaned)} rows remaining.")

except FileNotFoundError:
    print("Error: reviews.csv not found. Please ensure the file is in the 'data/' directory.")

Dataset loaded successfully with 23486 rows.
Starting data cleaning...
Data cleaned and sanitized. 22628 rows remaining.


## 3. Sample Creation
To ensure rapid development and testing, we create two distinct samples:
* A **small sample (100 reviews)** for fast, iterative tests on row-level models.
* A **larger sample (2000 reviews)** for Topic Modeling, which requires a larger corpus to generate meaningful results.

In [11]:
if 'df_cleaned' in locals() and not df_cleaned.empty:
    # Small sample for fast, iterative tests
    df_sample_fast = df_cleaned.sample(100, random_state=42).copy()
    print(f"Created 'df_sample_fast' with {len(df_sample_fast)} reviews for rapid testing.")

    # Larger sample required for meaningful Topic Modeling
    df_sample_topic = df_cleaned.sample(2000, random_state=42).copy()
    print(f"Created 'df_sample_topic' with {len(df_sample_topic)} reviews for Topic Modeling.")

Created 'df_sample_fast' with 100 reviews for rapid testing.
Created 'df_sample_topic' with 2000 reviews for Topic Modeling.


## 4. AI Model Development & Validation

### 4.1. Sentiment Analysis
* **Business Question:** Are our customers generally happy or unhappy?
* **Model Chosen:** `distilbert-base-uncased-finetuned-sst-2-english`
* **Justification:** Chosen for its excellent balance of speed and accuracy, making it ideal for scalable, serverless environments.

In [12]:
print("Loading Sentiment Analysis model...")
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

def safe_get_sentiment(text):
    """Applies sentiment analysis with error handling."""
    try:
        return sentiment_pipeline(text[:512])[0]['label']
    except Exception as e:
        print(f"--> SENTIMENT FAILED on review: '{text[:50]}...' | Error: {e}")
        return "ERROR"

print(f"Applying Sentiment Analysis to the fast sample ({len(df_sample_fast)} reviews)...")
df_sample_fast['sentiment_prediction'] = df_sample_fast['full_review_text'].progress_apply(safe_get_sentiment)
print("Sentiment Analysis complete.")

# --- Quantitative Validation ---
print("\n--- Model Validation ---")
# Define "ground truth" based on user ratings (e.g., > 3 stars is positive)
df_sample_fast['true_sentiment'] = np.where(df_sample_fast['Rating'] > 3, 'POSITIVE', 'NEGATIVE')

# Calculate accuracy
accuracy = accuracy_score(df_sample_fast['true_sentiment'], df_sample_fast['sentiment_prediction'])
print(f"Sentiment Model Accuracy on Sample: {accuracy:.2%}")

# Display a detailed classification report
print("\nClassification Report:")
print(classification_report(df_sample_fast['true_sentiment'], df_sample_fast['sentiment_prediction']))

Loading Sentiment Analysis model...
Applying Sentiment Analysis to the fast sample (100 reviews)...


100%|██████████| 100/100 [00:02<00:00, 42.96it/s]


Sentiment Analysis complete.

--- Model Validation ---
Sentiment Model Accuracy on Sample: 87.00%

Classification Report:
              precision    recall  f1-score   support

    NEGATIVE       0.68      0.88      0.76        24
    POSITIVE       0.96      0.87      0.91        76

    accuracy                           0.87       100
   macro avg       0.82      0.87      0.84       100
weighted avg       0.89      0.87      0.88       100



### 4.2. Zero-Shot Classification
* **Business Question:** What specific topics (e.g., 'price', 'shipping') are customers talking about?
* **Model Chosen:** `facebook/bart-large-mnli`
* **Justification:** This state-of-the-art model allows for dynamic classification against arbitrary labels, providing a powerful tool for interactive hypothesis testing.

In [13]:
print("Loading Zero-Shot Classification model...")
zero_shot_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

candidate_labels = ['price', 'quality', 'shipping', 'customer service', 'fit', 'fabric']

def safe_get_top_topic(review_text):
    """Applies zero-shot classification with error handling."""
    try:
        return zero_shot_classifier(review_text, candidate_labels)['labels'][0]
    except Exception as e:
        print(f"--> ZERO-SHOT FAILED on review: '{review_text[:50]}...' | Error: {e}")
        return "ERROR"

print(f"Applying Zero-Shot Classification to the fast sample ({len(df_sample_fast)} reviews)...")
df_sample_fast['zero_shot_topic'] = df_sample_fast['full_review_text'].progress_apply(safe_get_top_topic)
print("Zero-Shot Classification complete.")

# --- Qualitative Validation ---
print("\n--- Qualitative Validation Examples ---")
display(df_sample_fast[['full_review_text', 'zero_shot_topic']].head())

Loading Zero-Shot Classification model...
Applying Zero-Shot Classification to the fast sample (100 reviews)...


100%|██████████| 100/100 [01:54<00:00,  1.15s/it]


Zero-Shot Classification complete.

--- Qualitative Validation Examples ---


,full_review_text,zero_shot_topic
8329,"Change armpits I love, love this dress except for the armpits. if they had just made the armpits...",fit
17943,"Awkward sweater I wanted this sweater to work but sadly it failed. first, the pink was way to sh...",fabric
2157,Best. tee. ever. Oh my! i love this tee. it is super soft. i love how it doesn't look like a sac...,quality
11456,Well-made but lacks structure I love the style of this swimsuit on the model. when i purchased i...,quality
14386,"Strangely cut Was super excited to try this on, but had to go up 3 sizes from my normal 6 and th...",quality


### 4.3. Aspect-Based Sentiment Analysis (ABSA)
* **Business Question:** When customers discuss a topic, what specific *features* do they like or dislike?
* **Model Chosen:** `pyabsa` library (pre-trained model)
* **Justification:** We use this specialized library to extract fine-grained sentiment towards specific product features (aspects), offering actionable insights for product development.

In [14]:
print("Loading Aspect-Based Sentiment Analysis model...")
aspect_extractor = ATEPCCheckpointManager.get_aspect_extractor(checkpoint='english')

@stopit.threading_timeoutable(default="TIMEOUT_ERROR")
def analyze_review_with_timeout(review_text):
    """Core ABSA logic with a timeout decorator."""
    result = aspect_extractor.extract_aspect(inference_source=[review_text], print_result=False)
    if not result[0]['aspect']:
        return "N/A"
    return ", ".join([f"{aspect[0]} ({aspect[1]})" for aspect in result[0]['aspect']])

def safe_get_aspects(review_text):
    """Wrapper function that calls the core logic with a timeout and general error handling."""
    try:
        return analyze_review_with_timeout(review_text, timeout=20)
    except Exception as e:
        print(f"--> ABSA FAILED on review: '{review_text[:50]}...' | Error: {e}")
        return "GENERAL_ERROR"

print(f"Applying ABSA to the fast sample ({len(df_sample_fast)} reviews)...")
df_sample_fast['aspects'] = df_sample_fast['full_review_text'].progress_apply(safe_get_aspects)
print("ABSA complete.")

# --- Qualitative Validation & Error Checking ---
print("\n--- Qualitative Validation Examples ---")
display(df_sample_fast[df_sample_fast['aspects'] != 'N/A'][['full_review_text', 'aspects']].head())

failed_reviews = df_sample_fast[df_sample_fast['aspects'].str.contains("ERROR")]
if not failed_reviews.empty:
    print(f"\nWARNING: {len(failed_reviews)} reviews failed or timed out during ABSA.")

Loading Aspect-Based Sentiment Analysis model...
[2025-10-07 17:36:29] (2.4.2) ********** Available ATEPC model checkpoints for Version:2.4.2 (this version) **********
[2025-10-07 17:36:29] (2.4.2) ********** Available ATEPC model checkpoints for Version:2.4.2 (this version) **********
[2025-10-07 17:36:29] (2.4.2) Downloading checkpoint:english 
[2025-10-07 17:36:29] (2.4.2) Notice: The pretrained model are used for testing, it is recommended to train the model on your own custom datasets
[2025-10-07 17:36:29] (2.4.2) Checkpoint already downloaded, skip
FindFile Warning --> multiple targets ['Aspect Term Extraction and Polarity Classification.FAST_LCF_ATEPC.result.json', 'checkpoints\\ATEPC_ENGLISH_CHECKPOINT\\fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43\\fast_lcf_atepc.args.txt', 'checkpoints\\ATEPC_ENGLISH_CHECKPOINT\\fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43\\fast_lcf_atepc.config', 'checkpoints\\ATEPC_ENGLISH_CHECKPOINT\\fast_lcf_atepc_Engl

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['mask_predictions.classifier.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.LayerNorm.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have b

Applying ABSA to the fast sample (100 reviews)...


  0%|          | 0/100 [00:00<?, ?it/s]

[2025-10-07 17:36:35] (2.4.2) The results of aspect term extraction have been saved in d:\PROJECTS\reviewlens-ai\notebooks\Aspect Term Extraction and Polarity Classification.FAST_LCF_ATEPC.result.json


...

100%|██████████| 100/100 [02:12<00:00,  1.15s/it]

[2025-10-07 17:38:47] (2.4.2) The results of aspect term extraction have been saved in d:\PROJECTS\reviewlens-ai\notebooks\Aspect Term Extraction and Polarity Classification.FAST_LCF_ATEPC.result.json


100%|██████████| 100/100 [02:13<00:00,  1.34s/it]


ABSA complete.

--- Qualitative Validation Examples ---


,full_review_text,aspects
8329,"Change armpits I love, love this dress except for the armpits. if they had just made the armpits...",h (e)
17943,"Awkward sweater I wanted this sweater to work but sadly it failed. first, the pink was way to sh...","p (i), b (a), k (n)"
11456,Well-made but lacks structure I love the style of this swimsuit on the model. when i purchased i...,"s (t), w (i), l (i)"
14386,"Strangely cut Was super excited to try this on, but had to go up 3 sizes from my normal 6 and th...","c (u), w (a), c (u)"
18681,"Feminine and clean Size 8 always, 36c and i have broad shoulders...i found this true to size. th...","S (i), s (i), c (o)"


### 4.4. Topic Modeling
* **Business Question:** What are the hidden, high-level themes of conversation across all reviews?
* **Model Chosen:** `bertopic`
* **Justification:** BERTopic leverages transformer embeddings to find semantically coherent topics, which are more interpretable than traditional methods.

In [15]:
print(f"Starting Topic Modeling on the larger sample ({len(df_sample_topic)} reviews)...")
docs = df_sample_topic['full_review_text'].tolist()

# Initialize and fit the BERTopic model
topic_model = BERTopic(language="english", verbose=False)
topics, probs = topic_model.fit_transform(docs)

print("Topic Modeling complete!")

# --- Qualitative Validation ---
print("\n--- Discovered Topics Summary ---")
# Display a summary of the most prominent topics. Topic -1 contains outliers and can be ignored.
display(topic_model.get_topic_info())

Starting Topic Modeling on the larger sample (2000 reviews)...
Topic Modeling complete!

--- Discovered Topics Summary ---


,Topic,Count,Name,Representation,Representative_Docs
0,-1,699,-1_the_it_and_is,"[the, it, and, is, this, to, in, dress, but, on]",[ This dress is perfect for spring and early summer. it has some weight to it and so it drapes b...
1,0,218,0_these_they_them_are,"[these, they, them, are, pants, jeans, and, the, fit, in]",[Amazing comfortable pants! I really didn't expect to like these pants as much as i do. i ordere...
2,1,133,1_sweater_it_the_and,"[sweater, it, the, and, this, is, in, to, of, but]",[Terrific sweater with great detail; runs large I ordered the sweater in a medium but it was too...
3,2,95,2_dress_it_this_the,"[dress, it, this, the, is, love, and, to, of, so]","[Beautiful dress! I have had my eye on this dress since the spring (in yellow), but it sold out ..."
4,3,93,3_dress_petite_the_and,"[dress, petite, the, and, it, is, this, for, but, am]","[Wonderful dress! I love this dress. i think the cut, fabric and embroidery are all flattering. ..."
5,4,79,4_top_it_is_this,"[top, it, is, this, and, the, to, very, of, with]",[ This top is so pretty. the fabric is absolutely gorgeous and the lace on the collar and back a...
6,5,62,5_jacket_coat_it_and,"[jacket, coat, it, and, this, is, the, with, to, like]",[Amazing soft jacket This jacket is so adorable on! i have no idea why retailer styled it with a...
7,6,60,6_shirt_the_it_this,"[shirt, the, it, this, of, and, is, white, on, cute]",[Flowy and casual This shirt is awesome. i absolutely love the style of this shirt! it's super f...
8,7,58,7_blouse_the_and_it,"[blouse, the, and, it, is, this, for, in, of, lovely]","[Gorgeous blouse This is a stunning blouse! you can't tell from the photo, but it looks very 193..."
9,8,56,8_skirt_the_and_it,"[skirt, the, and, it, this, to, is, of, as, very]",[ Pros. 1) the color and texture of this skirt are gorgeous. twice as pretty as the photo .\r\n2...


## 5. Final Result & Conclusion
The sample DataFrame is now fully enriched with insights from all AI models. This validated logic is ready to be integrated into the production Lambda functions. The final data structure provides a multi-dimensional view of each customer review, enabling a rich, interactive analysis on the final dashboard.

In [16]:
# Select and reorder columns for a clean final view
# Note: 'bertopic_id' is not in this fast sample; it's a corpus-level insight.
final_columns = [
    'full_review_text', 
    'sentiment_prediction', 
    'zero_shot_topic',
    'aspects',
    'Rating',
    'true_sentiment'
]

print("--- Final Enriched Sample DataFrame (from fast sample) ---")
display(df_sample_fast[final_columns].head(10))

--- Final Enriched Sample DataFrame (from fast sample) ---


,full_review_text,sentiment_prediction,zero_shot_topic,aspects,Rating,true_sentiment
8329,"Change armpits I love, love this dress except for the armpits. if they had just made the armpits...",NEGATIVE,fit,h (e),4,POSITIVE
17943,"Awkward sweater I wanted this sweater to work but sadly it failed. first, the pink was way to sh...",NEGATIVE,fabric,"p (i), b (a), k (n)",2,NEGATIVE
2157,Best. tee. ever. Oh my! i love this tee. it is super soft. i love how it doesn't look like a sac...,POSITIVE,quality,N/A,5,POSITIVE
11456,Well-made but lacks structure I love the style of this swimsuit on the model. when i purchased i...,NEGATIVE,quality,"s (t), w (i), l (i)",3,NEGATIVE
14386,"Strangely cut Was super excited to try this on, but had to go up 3 sizes from my normal 6 and th...",NEGATIVE,quality,"c (u), w (a), c (u)",2,NEGATIVE
18681,"Feminine and clean Size 8 always, 36c and i have broad shoulders...i found this true to size. th...",POSITIVE,fit,"S (i), s (i), c (o)",5,POSITIVE
4124,"So comfortable I love the style of this dress, comfortable but chic, and it feels great!",POSITIVE,quality,s (t),5,POSITIVE
7991,Perfect lwd! This is perfect! it fits tts. i am usually an 8 and this was spot on. it isn't too ...,POSITIVE,fit,"l (w), f (i), h (u)",5,POSITIVE
8409,Yessssss!!!!! A culotte and basketball short hybrid. these culottes are heaven....very comfortab...,POSITIVE,quality,"c (u), w (a), b (a)",5,POSITIVE
21423,Love the color Love the color and the design. it is slightly see-through which i don't like but ...,POSITIVE,quality,"c (o), c (o), d (e)",5,POSITIVE
